# 作业

- 补全程序中的代码，理解其含义，并跑通整个项目；
- 报名参加[千言数据集：信息抽取比赛](https://aistudio.baidu.com/aistudio/competition/detail/46)。

![](https://ai-studio-static-online.cdn.bcebos.com/8747d266229c4ed0a3d410ea47a082f2551a0a3364464d5aae02047f151fa7f9)


# 基于预训练模型完成实体关系抽取

信息抽取旨在从非结构化自然语言文本中提取结构化知识，如实体、关系、事件等。对于给定的自然语言句子，根据预先定义的schema集合，抽取出所有满足schema约束的SPO三元组。

例如，「妻子」关系的schema定义为：      
{      
    S_TYPE: 人物,        
    P: 妻子,      
    O_TYPE: {      
        @value: 人物       
    }       
}        

该示例展示了如何使用PaddleNLP快速完成实体关系抽取，参与[千言信息抽取-关系抽取比赛](https://aistudio.baidu.com/aistudio/competition/detail/46)打榜。




In [1]:
# 安装paddlenlp最新版本
!pip install --upgrade paddlenlp

%cd relation_extraction/

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 430kB 15.7MB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.1
    Uninstalling paddlenlp-2.0.1:
      Successfully uninstalled paddlenlp-2.0.1
/home/aistudio/relation_extraction


## 关系抽取介绍

针对 DuIE2.0 任务中多条、交叠SPO这一抽取目标，比赛对标准的 'BIO' 标注进行了扩展。
对于每个 token，根据其在实体span中的位置（包括B、I、O三种），我们为其打上三类标签，并且根据其所参与构建的predicate种类，将 B 标签进一步区分。给定 schema 集合，对于 N 种不同 predicate，以及头实体/尾实体两种情况，我们设计对应的共 2*N 种 B 标签，再合并 I 和 O 标签，故每个 token 一共有 (2*N+2) 个标签，如下图所示。


<div align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/f984664777b241a9b43ef843c9b752f33906c8916bc146a69f7270b5858bee63" width="500" height="400" alt="标注策略" align=center />
</div>

### 评价方法

对测试集上参评系统输出的SPO结果和人工标注的SPO结果进行精准匹配，采用F1值作为评价指标。注意，对于复杂O值类型的SPO，必须所有槽位都精确匹配才认为该SPO抽取正确。针对部分文本中存在实体别名的问题，使用百度知识图谱的别名词典来辅助评测。F1值的计算方式如下：

F1 = (2 * P * R) / (P + R)，其中

- P = 测试集所有句子中预测正确的SPO个数 / 测试集所有句子中预测出的SPO个数
- R = 测试集所有句子中预测正确的SPO个数 / 测试集所有句子中人工标注的SPO个数

### Step1：构建模型

该任务可以看作一个序列标注任务，所以基线模型采用的是ERNIE序列标注模型。

**PaddleNLP提供了ERNIE预训练模型常用序列标注模型，可以通过指定模型名字完成一键加载。PaddleNLP为了方便用户处理数据，内置了对于各个预训练模型对应的Tokenizer，可以完成文本token化，转token ID，文本长度截断等操作。**

文本数据处理直接调用tokenizer即可输出模型所需输入数据。



In [2]:
import os
import json
from paddlenlp.transformers import ErnieForTokenClassification, ErnieTokenizer

label_map_path = os.path.join('data', "predicate2id.json")

if not (os.path.exists(label_map_path) and os.path.isfile(label_map_path)):
    sys.exit("{} dose not exists or is not a file.".format(label_map_path))
with open(label_map_path, 'r', encoding='utf8') as fp:
    label_map = json.load(fp)
    
num_classes = (len(label_map.keys()) - 2) * 2 + 2
print(num_classes)
# 补齐代码，理解TokenClassification接口含义，理解关系抽取标注体系和类别数由来
model = ErnieForTokenClassification.from_pretrained("ernie-1.0", num_classes=(len(label_map.keys()) - 2) * 2 + 2)
tokenizer = ErnieTokenizer.from_pretrained("ernie-1.0")

inputs = tokenizer(text="请输入测试样例", max_seq_len=20)

112


[2021-06-14 21:27:23,336] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-1.0
[2021-06-14 21:27:23,339] [    INFO] - Downloading ernie_v1_chn_base.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/ernie_v1_chn_base.pdparams
  0%|          | 0/392507 [00:00<?, ?it/s]100%|██████████| 392507/392507 [00:09<00:00, 40076.24it/s]
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.weight. classifier.weight is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/layers.py:1297: UserWarning: Skip loading for classifier.bias. classifier.bias is not found in the provided dict.
  warnings.warn(("Skip loading for {}. ".format(key) + str(err)))

In [3]:
print(tokenizer(text="吴宗宪遭服务生种族歧视, 他气呛", max_seq_len=20))

{'input_ids': [1, 1167, 761, 2075, 1396, 231, 112, 21, 106, 495, 2752, 367, 30, 44, 266, 5706, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


### Step2：加载并处理数据


从比赛官网下载数据集，解压存放于data/目录下并重命名为train_data.json, dev_data.json, test_data.json.

我们可以加载自定义数据集。通过继承[`paddle.io.Dataset`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/io/Dataset_cn.html#dataset)，自定义实现`__getitem__` 和 `__len__`两个方法。


In [4]:
from typing import Optional, List, Union, Dict

import numpy as np
import paddle
from tqdm import tqdm
from paddlenlp.utils.log import logger

from data_loader import parse_label, DataCollator, convert_example_to_feature
from extract_chinese_and_punct import ChineseAndPunctuationExtractor


class DuIEDataset(paddle.io.Dataset):
    """
    Dataset of DuIE.
    """

    def __init__(
            self,
            input_ids: List[Union[List[int], np.ndarray]],
            seq_lens: List[Union[List[int], np.ndarray]],
            tok_to_orig_start_index: List[Union[List[int], np.ndarray]],
            tok_to_orig_end_index: List[Union[List[int], np.ndarray]],
            labels: List[Union[List[int], np.ndarray, List[str], List[Dict]]]):
        super(DuIEDataset, self).__init__()

        self.input_ids = input_ids
        self.seq_lens = seq_lens
        self.tok_to_orig_start_index = tok_to_orig_start_index
        self.tok_to_orig_end_index = tok_to_orig_end_index
        self.labels = labels

    def __len__(self):
        if isinstance(self.input_ids, np.ndarray):
            return self.input_ids.shape[0]
        else:
            return len(self.input_ids)

    def __getitem__(self, item):
        return {
            "input_ids": np.array(self.input_ids[item]),
            "seq_lens": np.array(self.seq_lens[item]),
            "tok_to_orig_start_index":
            np.array(self.tok_to_orig_start_index[item]),
            "tok_to_orig_end_index": np.array(self.tok_to_orig_end_index[item]),
            # If model inputs is generated in `collate_fn`, delete the data type casting.
            "labels": np.array(
                self.labels[item], dtype=np.float32),
        }

    @classmethod
    def from_file(cls,
                  file_path: Union[str, os.PathLike],
                  tokenizer: ErnieTokenizer,
                  max_length: Optional[int]=512,
                  pad_to_max_length: Optional[bool]=None):
        assert os.path.exists(file_path) and os.path.isfile(
            file_path), f"{file_path} dose not exists or is not a file."
        label_map_path = os.path.join(
            os.path.dirname(file_path), "predicate2id.json")
        assert os.path.exists(label_map_path) and os.path.isfile(
            label_map_path
        ), f"{label_map_path} dose not exists or is not a file."
        with open(label_map_path, 'r', encoding='utf8') as fp:
            label_map = json.load(fp)
        chineseandpunctuationextractor = ChineseAndPunctuationExtractor()

        input_ids, seq_lens, tok_to_orig_start_index, tok_to_orig_end_index, labels = (
            [] for _ in range(5))
        dataset_scale = sum(1 for line in open(file_path, 'r'))
        logger.info("Preprocessing data, loaded from %s" % file_path)
        with open(file_path, "r", encoding="utf-8") as fp:
            lines = fp.readlines()
            for line in tqdm(lines):
                example = json.loads(line)
                input_feature = convert_example_to_feature(
                    example, tokenizer, chineseandpunctuationextractor,
                    label_map, max_length, pad_to_max_length)
                input_ids.append(input_feature.input_ids)
                seq_lens.append(input_feature.seq_len)
                tok_to_orig_start_index.append(
                    input_feature.tok_to_orig_start_index)
                tok_to_orig_end_index.append(
                    input_feature.tok_to_orig_end_index)
                labels.append(input_feature.labels)

        return cls(input_ids, seq_lens, tok_to_orig_start_index,
                   tok_to_orig_end_index, labels)


In [6]:
data_path = 'data'
batch_size = 64
max_seq_length = 128

train_file_path = os.path.join(data_path, 'train_data.json')
train_dataset = DuIEDataset.from_file(
    train_file_path, tokenizer, max_seq_length, True)
train_batch_sampler = paddle.io.BatchSampler(
    train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
collator = DataCollator()
train_data_loader = paddle.io.DataLoader(
    dataset=train_dataset,
    batch_sampler=train_batch_sampler,
    collate_fn=collator)

eval_file_path = os.path.join(data_path, 'dev_data.json')
test_dataset = DuIEDataset.from_file(
    eval_file_path, tokenizer, max_seq_length, True)
test_batch_sampler = paddle.io.BatchSampler(
    test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)
test_data_loader = paddle.io.DataLoader(
    dataset=test_dataset,
    batch_sampler=test_batch_sampler,
    collate_fn=collator)

[2021-06-14 21:28:21,120] [    INFO] - Preprocessing data, loaded from data/train_data.json
100%|██████████| 10010/10010 [00:20<00:00, 493.08it/s]
[2021-06-14 21:28:41,485] [    INFO] - Preprocessing data, loaded from data/dev_data.json
100%|██████████| 1000/1000 [00:01<00:00, 571.23it/s]


### Step3：定义损失函数和优化器，开始训练

我们选择均方误差作为损失函数，使用[`paddle.optimizer.AdamW`](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/optimizer/adamw/AdamW_cn.html#adamw)作为优化器。



在训练过程中，模型保存在当前目录checkpoints文件夹下。同时在训练的同时使用官方评测脚本进行评估，输出P/R/F1指标。
在验证集上F1可以达到69.42。


In [7]:
import paddle.nn as nn

class BCELossForDuIE(nn.Layer):
    def __init__(self, ):
        super(BCELossForDuIE, self).__init__()
        self.criterion = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, labels, mask):
        loss = self.criterion(logits, labels)
        mask = paddle.cast(mask, 'float32')
        loss = loss * mask.unsqueeze(-1)
        loss = paddle.sum(loss.mean(axis=2), axis=1) / paddle.sum(mask, axis=1)
        loss = loss.mean()
        return loss

In [8]:
from utils import write_prediction_results, get_precision_recall_f1, decoding

@paddle.no_grad()
def evaluate(model, criterion, data_loader, file_path, mode):
    """
    mode eval:
    eval on development set and compute P/R/F1, called between training.
    mode predict:
    eval on development / test set, then write predictions to \
        predict_test.json and predict_test.json.zip \
        under /home/aistudio/relation_extraction/data dir for later submission or evaluation.
    """
    example_all = []
    with open(file_path, "r", encoding="utf-8") as fp:
        for line in fp:
            example_all.append(json.loads(line))
    id2spo_path = os.path.join(os.path.dirname(file_path), "id2spo.json")
    with open(id2spo_path, 'r', encoding='utf8') as fp:
        id2spo = json.load(fp)

    model.eval()
    loss_all = 0
    eval_steps = 0
    formatted_outputs = []
    current_idx = 0
    for batch in tqdm(data_loader, total=len(data_loader)):
        eval_steps += 1
        input_ids, seq_len, tok_to_orig_start_index, tok_to_orig_end_index, labels = batch
        logits = model(input_ids=input_ids)
        mask = (input_ids != 0).logical_and((input_ids != 1)).logical_and((input_ids != 2))
        loss = criterion(logits, labels, mask)
        loss_all += loss.numpy().item()
        probs = F.sigmoid(logits)
        logits_batch = probs.numpy()
        seq_len_batch = seq_len.numpy()
        tok_to_orig_start_index_batch = tok_to_orig_start_index.numpy()
        tok_to_orig_end_index_batch = tok_to_orig_end_index.numpy()
        formatted_outputs.extend(decoding(example_all[current_idx: current_idx+len(logits)],
                                          id2spo,
                                          logits_batch,
                                          seq_len_batch,
                                          tok_to_orig_start_index_batch,
                                          tok_to_orig_end_index_batch))
        current_idx = current_idx+len(logits)
    loss_avg = loss_all / eval_steps
    print("eval loss: %f" % (loss_avg))

    if mode == "predict":
        predict_file_path = os.path.join("/home/aistudio/relation_extraction/data", 'predictions.json')
    else:
        predict_file_path = os.path.join("/home/aistudio/relation_extraction/data", 'predict_eval.json')

    predict_zipfile_path = write_prediction_results(formatted_outputs,
                                                    predict_file_path)

    if mode == "eval":
        precision, recall, f1 = get_precision_recall_f1(file_path,
                                                        predict_zipfile_path)
        os.system('rm {} {}'.format(predict_file_path, predict_zipfile_path))
        return precision, recall, f1
    elif mode != "predict":
        raise Exception("wrong mode for eval func")

In [9]:
from paddlenlp.transformers import LinearDecayWithWarmup

#learning_rate = 2e-5
learning_rate = 2e-4
num_train_epochs = 5
warmup_ratio = 0.06

criterion = BCELossForDuIE()
# Defines learning rate strategy.
steps_by_epoch = len(train_data_loader)
num_training_steps = steps_by_epoch * num_train_epochs
lr_scheduler = LinearDecayWithWarmup(learning_rate, num_training_steps, warmup_ratio)
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    apply_decay_param_fun=lambda x: x in [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])])

In [10]:
# 模型参数保存路径
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


### Step4：提交预测结果

加载训练保存的模型加载后进行预测。

**NOTE:** 注意设置用于预测的模型参数路径。

In [11]:
import time
import paddle.nn.functional as F

# Starts training.
global_step = 0
logging_steps = 50
save_steps = 10000
num_train_epochs = 2
output_dir = 'checkpoints'
tic_train = time.time()
model.train()
for epoch in range(num_train_epochs):
    print("\n=====start training of %d epochs=====" % epoch)
    tic_epoch = time.time()
    for step, batch in enumerate(train_data_loader):
        input_ids, seq_lens, tok_to_orig_start_index, tok_to_orig_end_index, labels = batch
        logits = model(input_ids=input_ids)
        mask = (input_ids != 0).logical_and((input_ids != 1)).logical_and(
            (input_ids != 2))
        loss = criterion(logits, labels, mask)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_gradients()
        loss_item = loss.numpy().item()

        if global_step % logging_steps == 0:
            print(
                "epoch: %d / %d, steps: %d / %d, loss: %f, speed: %.2f step/s"
                % (epoch, num_train_epochs, step, steps_by_epoch,
                    loss_item, logging_steps / (time.time() - tic_train)))
            tic_train = time.time()

        if global_step % save_steps == 0 and global_step != 0:
            print("\n=====start evaluating ckpt of %d steps=====" %
                    global_step)
            precision, recall, f1 = evaluate(
                model, criterion, test_data_loader, eval_file_path, "eval")
            print("precision: %.2f\t recall: %.2f\t f1: %.2f\t" %
                    (100 * precision, 100 * recall, 100 * f1))
            print("saving checkpoing model_%d.pdparams to %s " %
                    (global_step, output_dir))
            paddle.save(model.state_dict(),
                        os.path.join(output_dir, 
                                        "model_%d.pdparams" % global_step))
            model.train()

        global_step += 1
    tic_epoch = time.time() - tic_epoch
    print("epoch time footprint: %d hour %d min %d sec" %
            (tic_epoch // 3600, (tic_epoch % 3600) // 60, tic_epoch % 60))

# Does final evaluation.
print("\n=====start evaluating last ckpt of %d steps=====" %
        global_step)
precision, recall, f1 = evaluate(model, criterion, test_data_loader,
                                    eval_file_path, "eval")
print("precision: %.2f\t recall: %.2f\t f1: %.2f\t" %
        (100 * precision, 100 * recall, 100 * f1))
paddle.save(model.state_dict(),
            os.path.join(output_dir,
                            "model_%d.pdparams" % global_step))
print("\n=====training complete=====")


=====start training of 0 epochs=====
epoch: 0 / 2, steps: 0 / 156, loss: 0.708722, speed: 66.17 step/s
epoch: 0 / 2, steps: 50 / 156, loss: 0.051778, speed: 2.51 step/s
epoch: 0 / 2, steps: 100 / 156, loss: 0.021021, speed: 2.50 step/s
epoch: 0 / 2, steps: 150 / 156, loss: 0.013120, speed: 2.50 step/s
epoch time footprint: 0 hour 1 min 2 sec

=====start training of 1 epochs=====
epoch: 1 / 2, steps: 44 / 156, loss: 0.010394, speed: 2.50 step/s
epoch: 1 / 2, steps: 94 / 156, loss: 0.008290, speed: 2.49 step/s
epoch: 1 / 2, steps: 144 / 156, loss: 0.007733, speed: 2.40 step/s
epoch time footprint: 0 hour 1 min 3 sec

=====start evaluating last ckpt of 312 steps=====


  0%|          | 0/15 [00:00<?, ?it/s]

111 [[0.79292583 0.17623125 0.00260663 ... 0.00198923 0.00223831 0.00204311]
 [0.99067605 0.0077444  0.00211638 ... 0.00125189 0.0019287  0.002126  ]
 [0.27155313 0.01013863 0.0058095  ... 0.00530959 0.00711821 0.00541888]
 ...
 [0.99106324 0.00841384 0.00217154 ... 0.00125049 0.00195815 0.00208309]
 [0.90263796 0.00737613 0.00214114 ... 0.00155687 0.00224502 0.00205512]
 [0.79292583 0.17623118 0.00260663 ... 0.00198923 0.00223831 0.00204311]]
111 [[0.70018    0.23845881 0.00281556 ... 0.00217912 0.00226507 0.00212392]
 [0.06173953 0.03593894 0.00621556 ... 0.00615661 0.00819748 0.0050516 ]
 [0.10407628 0.8719081  0.00351327 ... 0.00189207 0.00216487 0.00274059]
 ...
 [0.9893385  0.00731503 0.00199978 ... 0.00126249 0.00192637 0.00193686]
 [0.98836225 0.00887519 0.0020442  ... 0.00121779 0.0018681  0.00192945]
 [0.9687479  0.00855871 0.00196069 ... 0.00130473 0.00180481 0.0016544 ]]
111 [[0.7893691  0.171614   0.00284542 ... 0.00213069 0.00239742 0.00215808]
 [0.9872048  0.00741154 0.0

  7%|▋         | 1/15 [00:00<00:06,  2.04it/s]

111 [[0.50220835 0.36554465 0.00310107 ... 0.00267788 0.00265643 0.00242989]
 [0.9907575  0.00684745 0.00205258 ... 0.0012896  0.00203062 0.00226444]
 [0.04276246 0.03435493 0.00849102 ... 0.00775787 0.0123292  0.00757471]
 ...
 [0.27557626 0.03331489 0.00311368 ... 0.00292639 0.00389953 0.00250655]
 [0.25603828 0.03189815 0.00323919 ... 0.00312049 0.00428811 0.00263663]
 [0.20104343 0.02998616 0.00392984 ... 0.00355753 0.00534273 0.00296495]]
111 [[0.7880434  0.17686534 0.00272727 ... 0.00204142 0.00226961 0.00208939]
 [0.12266693 0.01806202 0.00606914 ... 0.00600238 0.00831541 0.00557359]
 [0.15360548 0.83367443 0.00329433 ... 0.00165396 0.00204954 0.00274181]
 ...
 [0.98819935 0.0072803  0.002084   ... 0.0012587  0.00189557 0.00183696]
 [0.99027133 0.00704013 0.00213044 ... 0.00129768 0.00191647 0.00195252]
 [0.98609734 0.00660515 0.0020934  ... 0.0012969  0.00184603 0.00176986]]
111 [[0.6709841  0.25996974 0.00279194 ... 0.00221543 0.00234005 0.002176  ]
 [0.9919869  0.00645959 0.0

...

111 [[0.6382986  0.2873754  0.00330661 ... 0.00252308 0.00256213 0.00248932]
 [0.11016931 0.01887749 0.0079676  ... 0.00756978 0.01136202 0.00686021]
 [0.08321656 0.8426114  0.00340342 ... 0.00208846 0.0022959  0.00298804]
 ...
 [0.69019884 0.01503864 0.0023854  ... 0.00220183 0.00265084 0.00187938]
 [0.83522    0.02269024 0.00204398 ... 0.00147861 0.00199904 0.00170385]
 [0.7688914  0.01864246 0.00234727 ... 0.00175902 0.00226668 0.0018378 ]]
111 [[0.7503191  0.20341346 0.0027105  ... 0.00205255 0.00218948 0.0021002 ]
 [0.99130124 0.00705684 0.00209319 ... 0.00125299 0.00195653 0.0020832 ]
 [0.99132204 0.00773947 0.00212502 ... 0.00123711 0.0019521  0.00210427]
 ...
 [0.9771311  0.00739866 0.00183275 ... 0.00125597 0.00186514 0.00184034]
 [0.9376717  0.01453853 0.00182149 ... 0.00122315 0.0018626  0.00174039]
 [0.97989756 0.01565906 0.0019619  ... 0.00112387 0.00174231 0.00179266]]
111 [[0.67616105 0.25634277 0.00293431 ... 0.00233392 0.00249518 0.00227216]
 [0.9916671  0.00627995 0.0

 93%|█████████▎| 14/15 [00:06<00:00,  2.26it/s]

111 [[0.8170255  0.15126964 0.00271387 ... 0.00204344 0.00234057 0.0020792 ]
 [0.99093527 0.00736508 0.00212708 ... 0.00124645 0.00196181 0.00216167]
 [0.99073225 0.00837762 0.00212315 ... 0.00121601 0.00197078 0.00218551]
 ...
 [0.99105775 0.00801605 0.00216083 ... 0.00122897 0.00196716 0.00211773]
 [0.99095917 0.00841486 0.00217468 ... 0.00122517 0.00198015 0.00211885]
 [0.8170255  0.15126961 0.00271387 ... 0.00204344 0.00234057 0.0020792 ]]
111 [[0.63756955 0.2890867  0.00309016 ... 0.00236088 0.00249877 0.00236419]
 [0.14109664 0.01245768 0.00794824 ... 0.00838922 0.01116418 0.00704399]
 [0.06061069 0.8713652  0.00362991 ... 0.00201384 0.00238953 0.00297756]
 ...
 [0.94873345 0.01858362 0.00191037 ... 0.00123476 0.00180628 0.00156302]
 [0.9504468  0.01698007 0.00191952 ... 0.0012501  0.00183756 0.00166341]
 [0.7627455  0.0073614  0.00242656 ... 0.00221645 0.00292437 0.00228634]]
111 [[0.59367883 0.28711414 0.00303777 ... 0.00245011 0.00254663 0.00233805]
 [0.99184024 0.0062494  0.0

100%|██████████| 15/15 [00:06<00:00,  2.33it/s]


eval loss: 0.007536
precision: 0.00	 recall: 0.00	 f1: 0.00	

=====training complete=====


In [12]:
!bash train.sh

In [13]:
!bash predict.sh

预测结果会被保存在data/predictions.json，data/predictions.json.zip，其格式与原数据集文件一致。

之后可以使用官方评估脚本评估训练模型在dev_data.json上的效果。如：

```shell
python re_official_evaluation.py --golden_file=dev_data.json  --predict_file=predicitons.json.zip [--alias_file alias_dict]
```
输出指标为Precision, Recall 和 F1，Alias file包含了合法的实体别名，最终评测的时候会使用，这里不予提供。

之后在test_data.json上预测，然后预测结果（.zip文件）至[千言评测页面](https://aistudio.baidu.com/aistudio/competition/detail/46)。





In [30]:
!python re_official_evaluation.py --golden_file=./data/dev_data.json  --predict_file=./data/duie.json.zip 

correct spo num = 0.0
submitted spo num = 0.0
golden set spo num = 1771.0
submitted recall spo num = 0.0
{"errorCode": 0, "errorMsg": "success", "data": [{"name": "precision", "value": 0.0}, {"name": "recall", "value": 0.0}, {"name": "f1-score", "value": 0.0}]}



## Tricks

### 尝试更多的预训练模型

基线采用的预训练模型为ERNIE，PaddleNLP提供了丰富的预训练模型，如BERT，RoBERTa，Electra，XLNet等
参考[预训练模型文档](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/transformers.html)

如可以选择RoBERTa large中文模型优化模型效果，只需更换模型和tokenizer即可无缝衔接。

In [ ]:
from paddlenlp.transformers import RobertaForTokenClassification, RobertaTokenizer

model = RobertaForTokenClassification.from_pretrained(
    "roberta-wwm-ext-large",
    num_classes=(len(label_map) - 2) * 2 + 2)
tokenizer = RobertaTokenizer.from_pretrained("roberta-wwm-ext-large")

### 模型集成

使用多个模型进行训练预测，将各个模型预测结果进行融合。

以上基线实现基于PaddleNLP，开源不易，希望大家多多支持~ 
**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐，及时跟踪最新消息和功能哦**

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
